In [ ]:
# Utilizzando il modulo random, realizza il gioco del sasso, carta, forbici:
# - sasso vince su forbici
# - carta vince su sasso
# - forbici vince su carta
# - ogni simbolo pareggia contro ogni simbolo uguale

# Implementate l'algoritmo:
# - definendo una funzione per giocare
# - permettendo all'utente di scegliere tra sasso, carta e forbici
# - registrando il numero di vittorie, pareggi e sconfitte
# - continuando a far giocare l'utente fino a quando non decide di smettere

# BONUS: registra i dati della partita su un file di testo, decidendo quale sia il formato migliore

# BONUS 2: prevedi e testa una funzione per poter leggere da un file di testo i dati di una partita e visualizzarli

# BONUS 3: i dati di ciascuna partita devono essere salvati in un file txt che riporta la data 
# e l'ora della fine della partita (ad esempio se la partita finisce il 14/06/2024 alle 20:02, dovrà chiamarsi
#                                   "2024_06_14_20_02.txt")

# BONUS 4: prevedi una funzione per poter accedere a tutti i dati delle partite salvate e fare una statistica dei giocatori.
#          Prevedi quindi di chiedere il nickname del giocatore prima di avviare la partita. Il nickname dovrà comparire 
#          tra i dati del .txt 

In [2]:
import datetime
import random
import pandas as pd
import os 

lista_figure = ["sasso", "carta", "forbici"]
dataCorrente = datetime.datetime.now()
data_str = dataCorrente.strftime('%m_%d_%y    %H_%M') ## perchè se utilizzo gli underscore per la data non posso utillizare i due punti per l'ora????
nome_file1=data_str+ ".csv"
nick=input("Inserisci il nickname") 

def gioco():
    vittorie = 0
    pareggi = 0
    sconfitte = 0
    continua = True  
    
    while continua:
        while True:
            scegli_simbolo = input("Scegli una figura tra carta, sasso e forbici: ")
            if scegli_simbolo in lista_figure:
                break
            else:
                print("Inserisci la figura corretta!")
        figura_computer = random.choice(lista_figure)
        print(f"Tu hai scelto: {scegli_simbolo}")
        print(f"Il computer ha scelto: {figura_computer}")
        if scegli_simbolo == figura_computer:
            print("Pareggio!")
            pareggi += 1
############  Metodo alternativo  spiegato in lezione ###############
            # #condizioniVittoria=[
            #     scegli_simbolo == "sasso" and figura_computer == "forbici"
            #     scegli_simbolo == "carta" and figura_computer == "sasso"
            #     scegli_simbolo == "forbici" and figura_computer == "carta"
            # ]
            
            # if any(condizioniVittoria)
            #print("Hai vinto")
               #else:
                    #print("Hai perso")
                    
#############################################################################

        elif (scegli_simbolo == "sasso" and figura_computer == "forbici") or \
             (scegli_simbolo == "carta" and figura_computer == "sasso") or \
             (scegli_simbolo == "forbici" and figura_computer == "carta"):
            print("Hai vinto!")
            vittorie += 1
        else:
            print("Hai perso!")
            sconfitte += 1
        while True:
######### walrus operator :=  in aggiunta ^40 (che significa x40 spazi)#######################
            messaggio=("Vuoi continuare? (si/no):")
            risposta = input(f"{messaggio:=^40}  ").lower()
            if risposta == "si":
                continua = True
                break
            elif risposta == "no":
                continua = False
                break
            else:
                print("Devi scegliere se continuare digitando 'si' o 'no'.")

    salva(nick + ".csv",data_str,vittorie, pareggi, sconfitte)
    df=leggi_csv(nick + ".csv")
    vittorie_totali=df["vittorie"].sum()
    sconfitte_totali=df["sconfitte"].sum()
    pareggi_totali=df["pareggi"].sum()
    
    #DataFrame con colonne nickname,vittorie totali, sconfitte totali e pareggi totali e come righe avra una riga per ogni utente
    #nickname, vittorie totali,sconfitte totale,pareggi totali
    #pippo, 10,2,3
    #pluto, 5,3,4
    aggiorna_classifica(path="classifica.csv",nick=nick,
                        vittorie_totali=vittorie_totali,
                        pareggi_totali=pareggi_totali,
                        sconfitte_totali=sconfitte_totali,)
    #ranking("classifica.csv",vittorie_totali)

def salva(path,data_str,vittorie, pareggi, sconfitte):
    data={
        "data":[data_str],
        "vittorie":[vittorie],
        "sconfitte":[sconfitte],
        "pareggi":[pareggi]
    }
    df=pd.DataFrame(data=data)
    df.set_index("data",inplace=True)
    if os.path.isfile(path=path):
        df.to_csv(path, mode="a",header=False)
    else:
        df.to_csv(path)        

def leggi_csv(path):
    df=pd.read_csv(filepath_or_buffer=path,sep=",")
    return df.iloc[-1] # prendo solo l'ultimo risultato e aggiungo a quello senno si cumulava un risultato in più

# def aggiorna_classifica(path,nick,vittorie_totali,pareggi_totali,sconfitte_totali):
#     data={
#         "nick":[],
#         "Vittorie Totali":[],
#         "Sconfitte Totali":[],
#         "Pareggi Totali":[]
#     }
#     df=pd.DataFrame(data=data)
#     df.set_index("nick",inplace=True)
#     df.loc[nick]=[vittorie_totali,sconfitte_totali,pareggi_totali]
#     if os.path.isfile(path=path) and data["nick"]== nick:
#         df.to_csv(path, mode="w", index=False,header=False)
#         data["nick"]=nick
#         data["Vittorie Totali"]=vittorie_totali
#         data["Sconfitte Totali"]=sconfitte_totali
#         data["Pareggi Totali"]=pareggi_totali
#     else:
#         df.to_csv(path,mode="a",header=True)

def aggiorna_classifica(path, nick, vittorie_totali, pareggi_totali, sconfitte_totali):
    if os.path.isfile(path):
        df = pd.read_csv(path, index_col="nick")
    else:
        data = {
            "nick": [],
            "Vittorie Totali": [],
            "Sconfitte Totali": [],
            "Pareggi Totali": []
        }
        df = pd.DataFrame(data=data)
#setto l'indice del dizionario come il nick 
        df.set_index("nick", inplace=True)
#qui controllo se il nick esiste negli indici gli aggiungo le vittorie, altrimento creo la riga(riga 127)
    if nick in df.index:
        df.loc[nick, "Vittorie Totali"] += vittorie_totali
        df.loc[nick, "Sconfitte Totali"] += sconfitte_totali
        df.loc[nick, "Pareggi Totali"] += pareggi_totali
#essendo che dovevo fare un altra funzione completamente uguale questa per leggere il csv rifarlo in data frame e fargli il max, 
# l'ho inserito direttamente alla fine dell'aggiornamento della classifica
        max_win=df["Vittorie Totali"].max()
        max_lose=df["Sconfitte Totali"].max()
        max_tie=df["Pareggi Totali"].max()
        print(f"Le vittorie totali sono : {max_win}, Le sconfitte totali sono: {max_lose}, I pareggi totali sono: {max_tie}")
    else:
        df.loc[nick] = [vittorie_totali, sconfitte_totali, pareggi_totali]
#ricopiato le funzioni per il ranking nel caso in cui il nick non esiste
        max_win=df["Vittorie Totali"].max()
        max_lose=df["Sconfitte Totali"].max()
        max_tie=df["Pareggi Totali"].max()
        print(f"Le vittorie totali sono : {max_win}, Le sconfitte totali sono: {max_lose}, I pareggi totali sono: {max_tie}")
        
 # Stampa dei primi dieci elementi della classifica
    df_sorted = df.sort_values(by=["Vittorie Totali", "Pareggi Totali", "Sconfitte Totali"], ascending=[False, False, True])##non sapevo della funzione sort,leggendola ho imparato l'ascending
    top_10 = df_sorted.head(10)
    
    for idx in range(len(top_10)):
        row = top_10.iloc[idx]
        print(f"posizione {idx + 1} {top_10.index[idx]}\n vittorie {row['Vittorie Totali']}\n sconfitte {row['Sconfitte Totali']}\n pareggi {row['Pareggi Totali']}")


    df.to_csv(path)



gioco()




    



Tu hai scelto: carta
Il computer ha scelto: carta
Pareggio!
Le vittorie totali sono : 1, Le sconfitte totali sono: 2, I pareggi totali sono: 2
posizione 1 pippo
 vittorie 1
 sconfitte 1
 pareggi 2
posizione 2 pluto
 vittorie 0
 sconfitte 0
 pareggi 1
posizione 3 paperino
 vittorie 0
 sconfitte 2
 pareggi 1
posizione 4 franco
 vittorie 0
 sconfitte 1
 pareggi 0
